In [58]:
# import dependencies
import pandas as pd
import json
import requests
from googleapiclient.discovery import build
from config import api_key
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [59]:
# Enter video id
# Snarky Puppy - Trinity
video_id = "iK8fezsgZYg"

In [60]:
def video_comments_to_df(video_id):
    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # retrieve youtube video results
    video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

    # create empty dataframe
    df = pd.DataFrame(columns=['id', 'author', 'thumbsup', 'text', '# of replies', 'replies'])

    # iterate video response
    while video_response:

    # export comments to a dataframe
        for i in range(len(video_response['items'])):
            replies = []
            id = video_response['items'][i]['snippet']['topLevelComment']['id']
            author = video_response['items'][i]['snippet']['topLevelComment']['snippet']['authorDisplayName']
            text = video_response['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal']
            replycount = video_response['items'][i]['snippet']['totalReplyCount']
            if replycount>0:
                for reply in video_response['items'][i]['replies']['comments']:
                    reply = reply['snippet']['textOriginal']
                    replies.append(reply)
            df = df.append({
                'id': id, 
                'author': author, 
                'thumbsup': int(video_response['items'][i]['snippet']['topLevelComment']['snippet']['likeCount']),
                'text': text, 
                '# of replies': replycount, 
                'replies': replies 
                }, 
                ignore_index=True)
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break

    # change data types
    df['thumbsup'] = df['thumbsup'].astype(int)
    df['# of replies'] = df['# of replies'].astype(int)

    return df

In [61]:
df = video_comments_to_df(video_id)

In [62]:
# export to csv
df.to_csv('comments.csv', index=False)


In [63]:
# total reply checker

# for "Trinity", total replies should be 1868
total_replies = len(df) + sum(df['# of replies'])
print(f'Total replies: {total_replies}')

# SUCCESS


Total replies: 277


In [64]:
print(len(df['author'].unique()))

190


In [65]:
df['author'].value_counts()

Josh Lampe           2
DamplyDoo            2
zeff trus            2
Mirkonin Hopsalot    1
devikwolf            1
                    ..
The Katmann          1
vinkuu               1
アミキニャン               1
Robby Genschel       1
TheNotSoCivilEngr    1
Name: author, Length: 190, dtype: int64

In [66]:
# get sentiment of comments

analyzer = SentimentIntensityAnalyzer()

In [67]:
comments = df['text'].to_list()
compound_values_list = []
for comment in comments:
    compound = analyzer.polarity_scores(comment)['compound']
    compound_values_list.append(compound)
df['compound'] = compound_values_list
print(df['compound'].mean())
print(len(df['compound'] ))

0.21182020725388603
193


In [68]:
df[df['compound'] < -0.5]

,id,author,thumbsup,text,# of replies,replies,compound
8,UgwroBbPs-Ca7mC7CEJ4AaABAg,Angel Gutierrez,0,I wonder what’s the percentage of people watch...,0,[],-0.5574
16,UgzpHIlWEGiqGzRLdVp4AaABAg,Tommi Raulahti,0,"Too bad, here in Finland, stores and every oth...",0,[],-0.5423
31,Ugzl746lnNmmpeIlj4F4AaABAg,LostTman1093,0,We just gonna ignore the fact he broke into hi...,0,[],-0.6868
33,UgwgxMSXgY39cIOC4N54AaABAg,Frazzled Haloz,0,Grrrrrr I HATE political ads….,0,[],-0.6633
51,Ugy0m-QyitIyd8u4qBN4AaABAg,Chris Cowling,0,Would order but covertinstruments only ships t...,0,[],-0.6271
69,UgyrYTWBmtvFNddMcrZ4AaABAg,Robby Genschel,0,"Damn, this is #4? I haven't been on the channe...",0,[],-0.6059
81,UgzZEdTswaxUpRt71Zx4AaABAg,Kayaking Rivers,0,Do you ever worry that the covert companion wi...,0,[],-0.7579
85,Ugw24u5BVTS3zmytn-F4AaABAg,Ian Brooke,0,But most stores are relying on alarms. The ea...,0,[],-0.8449
104,UgyeqtxrLA5pIBEuicJ4AaABAg,KlejnotNilu,11,I've heard that fifth expansion gonna contain ...,3,[Pocket thermite charge. Can't be locked if it...,-0.5574
116,Ugw_jMpFOCTuIDAgDv14AaABAg,Brian Posey,0,2 years old? damn how time flys!! can't believ...,0,[],-0.5080
